In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "CPU"
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
tf.get_logger().setLevel(tf.compat.v1.logging.ERROR)

In [ ]:
from sklearn import *

In [ ]:
## packages
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn import metrics
import lightgbm as lgb
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve,accuracy_score,log_loss,accuracy_score,f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from mlxtend.classifier import StackingCVClassifier
from sklearn.metrics import roc_auc_score
from sklearn.neural_network import MLPClassifier
from sklearn import svm
from keras.callbacks import EarlyStopping
import sklearn
import mlxtend
import keras
from scikeras.wrappers import KerasClassifier
#from keras.wrappers.scikit_learn import KerasClassifier
import matplotlib.pyplot as plt

In [ ]:
%%time
df=pd.read_csv('/home/vinod/asetiya/train_test_data_10449_bonferroni.csv')
df

In [ ]:
df.fillna(0,inplace=True)
df.replace([np.inf, -np.inf], 0, inplace=True)

In [ ]:
np.random.seed(42)
y=df['Toxicity']
X=df.drop(['Toxicity'],axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
np.random.seed(42)
from sklearn.preprocessing import StandardScaler

# Select only the numerical columns
numerical_columns = X.columns[(X.dtypes == 'float64') & (X.columns != 'IPC')]


# Create a DataFrame with only the numerical columns
numerical_X = X[numerical_columns]

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler on the numerical data and transform it for the train set
scaled_numerical_X_train = scaler.fit_transform(numerical_X.loc[X_train.index])

# Replace the scaled numerical data back into the original DataFrame for the train set
X.loc[X_train.index, numerical_columns] = scaled_numerical_X_train

# Transform the numerical data for the test set
scaled_numerical_X_test = scaler.transform(numerical_X.loc[X_test.index])

# Replace the scaled numerical data back into the original DataFrame for the test set
X.loc[X_test.index, numerical_columns] = scaled_numerical_X_test


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import random as rn
rn.seed(12345)

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)
# Initialize the base models
model_lgb = lgb.LGBMClassifier(colsample_bytree= 0.5,
 lambda_l2= 0.8, 
 learning_rate= 0.01,
 max_depth= 10,
 min_child_samples= 32,
 min_child_weight= 0.01,
 num_leaves= 40,
 subsample= 0.84,
 objective= 'binary',
 metric='auc',
 boosting_type= 'dart',
 is_unbalance= True,random_state=34,
 boost_from_average= False,n_estimators=100)



model_rf = RandomForestClassifier(criterion= 'entropy',max_depth= 12,
                                  min_samples_leaf= 11,min_samples_split= 39,n_estimators= 300,random_state=34,class_weight='balanced')  



# Define the TensorFlow model
def create_tf_model():
    tf.random.set_seed(89)
    model = tf.keras.Sequential([
        
    tf.keras.layers.Dense(1024, activation='relu', input_shape=(X_train.shape[1],), 
                              kernel_regularizer=tf.keras.regularizers.l2(0.00803)),
    tf.keras.layers.Dropout(0.306, seed=12),
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.00803)),
    tf.keras.layers.Dropout(0.306, seed=12),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.00803)),
    tf.keras.layers.Dropout(0.306, seed=12),
    tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Create a KerasClassifier from the TensorFlow model
model_tf = KerasClassifier(build_fn=create_tf_model, epochs=10, batch_size=32,class_weight="balanced")  



In [ ]:

# # Initialize the meta-learner
meta_learner = LogisticRegression(class_weight="balanced")  
np.random.seed(42)

# Define the stacking classifier with stratified k-fold CV
stacked_model = StackingCVClassifier(classifiers=[model_lgb, model_rf, model_tf],
                                     meta_classifier=meta_learner,
                                     cv=StratifiedKFold(n_splits=5),
                                     use_probas=True, random_state=34)

# Train the stacking classifier on the training data
stacked_model.fit(X_train, y_train)

# Make predictions on the training set
train_predictions = stacked_model.predict(X_train)

# Make predictions on the test set
test_predictions = stacked_model.predict(X_test)

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, roc_curve, auc, precision_recall_curve
from sklearn.metrics import matthews_corrcoef
# Calculate metrics for the training set
train_accuracy = accuracy_score(y_train, train_predictions)
train_f1 = f1_score(y_train, train_predictions)
train_mcc = matthews_corrcoef(y_train, train_predictions)
train_confusion_matrix = confusion_matrix(y_train, train_predictions)

# Calculate metrics for the test set
test_accuracy = accuracy_score(y_test, test_predictions)
test_f1 = f1_score(y_test, test_predictions)
test_mcc = matthews_corrcoef(y_test, test_predictions)
test_confusion_matrix = confusion_matrix(y_test, test_predictions)

# Calculate AUROC using cross-validated predictions
cross_val_predictions = stacked_model.predict_proba(X_train)[:, 1]
cross_val_auroc = roc_auc_score(y_train, cross_val_predictions)

# Print the cross-validated AUROC and metrics
print("Cross-Validated AUROC:", cross_val_auroc)

print("Train Metrics:")
print("Accuracy:", train_accuracy)
print("F1 Score:", train_f1)
print("MCC:", train_mcc)
print("Confusion Matrix:\n", train_confusion_matrix)

cross_val_predictions_test = stacked_model.predict_proba(X_test)[:, 1]
cross_val_auroc_test = roc_auc_score(y_test, cross_val_predictions_test)

print("Test Metrics:")
# Print the cross-validated AUROC and metrics
print("Cross-Validated AUROC:", cross_val_auroc_test)
print("Accuracy:", test_accuracy)
print("F1 Score:", test_f1)
print("MCC:", test_mcc)
print("Confusion Matrix:\n", test_confusion_matrix)

In [ ]:
np.random.seed(42)
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import pandas as pd

import matplotlib.pyplot as plt

np.random.seed(42)
# Define the models
models = [model_rf, model_tf, model_lgb]
labels = ['Random Forest', 'Multi layer Perceptron', 'LightGBM']

# Define the number of folds for cross-validation
n_splits = 5
auc_scores_train = []
auc_scores_val = []
auc_scores_test = []


# Iterate over models
for model, lab in zip(models, labels):
    skf = StratifiedKFold(n_splits=n_splits, random_state=34, shuffle=True)
    auc_scores_val = []  # Reset the list for validation AUC scores
    auc_scores_test_model = []  # Separate list for test AUC scores
    auc_scores_train_model = []
    
    # Perform stratified k-fold cross-validation
    for train_index, val_index in skf.split(X_train, y_train):
        X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]
        
        # Train the model on the current fold
        model.fit(X_train_fold, y_train_fold)
        
        # Predict probabilities on the training set using the individual model
        train_predictions_proba = model.predict_proba(X_train_fold)[:, 1]
        auc_train = roc_auc_score(y_train_fold, train_predictions_proba)
        auc_scores_train_model.append(auc_train)
        
        # Predict probabilities on the validation fold
        val_predictions = model.predict_proba(X_val_fold)[:, 1]
        auc_val = roc_auc_score(y_val_fold, val_predictions)
        auc_scores_val.append(auc_val)
        
    # Predict probabilities on the test set using the individual model
    test_predictions_proba = model.predict_proba(X_test)[:, 1]
    auc_test = roc_auc_score(y_test, test_predictions_proba)
    auc_scores_test_model.append(auc_test)
    
    # Calculate average AUC accuracy across all folds
    average_auc_train = np.mean(auc_scores_train_model)
    average_auc_val = np.mean(auc_scores_val)
    average_auc_test = np.mean(auc_scores_test_model)
    print(f"{lab} Average AUC _TRAIN: {average_auc_train:.4f}")
    print(f"{lab} Average AUC _VAL: {average_auc_val:.4f}")
    print(f"{lab} Average AUC _TEST: {average_auc_test:.4f}")
    
    auc_scores_test.append(auc_scores_test_model)  # Append the list for each model
    auc_scores_train.append(auc_scores_train_model)
    auc_scores_val.append(auc_scores_val)
    

In [ ]:
auc_scores_test
labels=['Random Forest', 'Multi layer Perceptron', 'LightGBM']
auc_scores = {'Model': labels, 'AUC Train': auc_scores_train, 'AUC Test': auc_scores_test,}
df_auc = pd.DataFrame(auc_scores)

## McNemar Test

In [ ]:
#Null hypotheis: there is no significant difference between performance of the two algorithm
#Alternate hypothesis, which argues that there is a significant difference in performance between the two algorithms

from mlxtend.evaluate import mcnemar_table
from mlxtend.evaluate import mcnemar

# Assuming y_test and y_pred are available for each model
# Replace 'model_rf', 'model_tf', 'model_lgb', and 'stacked_model' with the actual variable names
models = [model_rf, model_tf, model_lgb, stacked_model]
labels = ['Random Forest', 'Multi-layer Perceptron', 'LightGBM', 'Stacked']

# Initialize an empty list to store the results
results = []

y_pred_stacked=stacked_model.predict(X_test)

# Iterate over each model
for model_index, model in enumerate(models):
    y_pred = model.predict(X_test)
    
    # Create the contingency table for McNemar's test
    table = mcnemar_table(y_target=y_test, y_model1=y_pred_stacked, y_model2=y_pred)
    
    # Print the table for reference
    print(f"\nContingency table for {labels[model_index]} vs Stacked Model:")
    print(table)
    
    # Perform McNemar's test
    chi2, p = mcnemar(ary=table, corrected=True)

    # Store the results
    results.append((labels[model_index], chi2, p))

# Display the results
print("\nMcNemar's Test Results:")
print("Model vs Stacked Model  Chi-squared  P-value")
for result in results:
    print(f"{result[0]:<24} {result[1]:<12.4f} {result[2]:<8.4f}")


## external validation

In [ ]:
df_sample=pd.read_csv("/home/vinod/asetiya/RSC_rev/extval_180.csv",index_col=0)
df_sample['Toxicity']

In [ ]:
df_sample=df_sample[df.columns]  #extracting relevant columns
df_sample

In [ ]:
df_sample.replace([np.inf, -np.inf], np.nan, inplace=True)
df_sample=df_sample.fillna(0)

In [ ]:
# Create X_sample
X_sample = df_sample[numerical_columns]
X_sample

In [ ]:

# Scale the numerical data in df_sample using the trained scaler
scaled_numerical_X_sample = scaler.transform(X_sample)

# Create a new DataFrame with the scaled numerical data
df_sample_scaled = pd.DataFrame(scaled_numerical_X_sample, columns=numerical_columns, index=X_sample.index)


In [ ]:
df_sample[numerical_columns] = df_sample_scaled
df_sample

In [ ]:
df_sample=df_sample.reindex(labels=df.columns,axis=1)

In [ ]:
x_sample=df_sample.drop('Toxicity',axis=1)

In [ ]:
predictions_ext = stacked_model.predict(x_sample)

In [ ]:
y_sample=df_sample['Toxicity']

In [ ]:
predictions_ext = stacked_model.predict(x_sample)
# Calculate accuracy
accuracy_ext = accuracy_score(y_sample, predictions_ext)
print(f"accuracy_ext:{accuracy_ext}")

# Calculate F1 score
f1_ext = f1_score(y_sample, predictions_ext)
print(f"f1_ext:{f1_ext}")
# Calculate precision
precision_ext = precision_score(y_sample, predictions_ext)
print(f"precision_ext: {precision_ext}")

# Calculate recall
recall_ext = recall_score(y_sample,predictions_ext)
print(f"recall_ext: {recall_ext}")


In [ ]:
pred_proba_ext = stacked_model.predict_proba(x_sample)[:, 1]

# Calculate ROC AUC
roc_auc_ext = roc_auc_score(y_sample, pred_proba_ext)
print(f"roc_auc_ext: {roc_auc_ext}")

# Confusion matrix
ext_confusion_matrix = confusion_matrix(y_sample,predictions_ext )
print("ext Confusion Matrix:")
print(ext_confusion_matrix)


In [ ]:
from sklearn.metrics import roc_auc_score

# Prepare your new dataset (X_sample, y_sample)

# Preprocess the new dataset

# Predict probabilities for the new dataset
y_pred_stacked = stacked_model.predict_proba(x_sample)[:, 1]

y_pred_rf = model_rf.predict_proba(x_sample)[:, 1]
y_pred_mlp = model_tf.predict_proba(x_sample)[:, 1]
y_pred_lgb = model_lgb.predict_proba(x_sample)[:, 1]

# Calculate AUC scores
auc_stacked = roc_auc_score(y_sample, y_pred_stacked)

auc_rf = roc_auc_score(y_sample, y_pred_rf)
auc_mlp = roc_auc_score(y_sample, y_pred_mlp)
auc_lgb = roc_auc_score(y_sample, y_pred_lgb)

# Print the AUC scores
print("Stacked Model AUC:", auc_stacked)
print("Random Forest AUC:", auc_rf)
print("Multi layer Perceptron AUC:", auc_mlp)
print("LightGBM AUC:", auc_lgb)


In [ ]:
## calibration plots


from sklearn.calibration import calibration_curve

models = [model_rf, model_tf, model_lgb, stacked_model]
labels = ['Random Forest', 'Multi-layer Perceptron', 'LightGBM', 'Stacked']

plt.figure(figsize=(8, 6))

for model, label in zip(models, labels):
    y_pred_ext = model.predict_proba(x_sample)[:, 1]
    fraction_of_positives, mean_predicted_value = calibration_curve(y_sample, y_pred_ext, n_bins=10)
    plt.plot(mean_predicted_value, fraction_of_positives, "s-", label=label)

plt.plot([0, 1], [0, 1], "k--", label="Perfectly calibrated")
plt.xlabel("Mean Predicted Value")
plt.ylabel("Fraction of Positives")
plt.title("Calibration Curve")
plt.legend(loc="lower right")
#plt.savefig('/home/md/asetiya/new_padel_fingerprints_10449/Calibration_ext.png')
#plt.show()


## etoxpred_probabilities 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the file from "etoxpred" containing molecule SMILES and toxicity probability scores
etoxpred_data = pd.read_csv("/home/vinod/asetiya/RSC_rev/share/results_smiles_test_etoxpred.csv")  # Replace 'etoxpred_results.csv' with the actual file name

# Process the file to extract the molecule SMILES and corresponding toxicity probability scores
etoxpred_probabilities = etoxpred_data['Tox-score']

# Predict toxicity probabilities using your model on the same set of molecules
moltoxpred_probabilities = test_predictions  

In [ ]:

# Load the file from "etoxpred" containing molecule SMILES and toxicity probability scores
etoxpred_data_ext= pd.read_csv("/home/vinod/asetiya/RSC_rev/share/results_smiles_ext_etoxpred.csv")  # Replace 'etoxpred_results.csv' with the actual file name

# Process the file to extract the molecule SMILES and corresponding toxicity probability scores
etoxpred_probabilities_ext = etoxpred_data_ext['Tox-score']
 


In [ ]:
import numpy as np
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

np.random.seed(42)

# Define the models
models = [model_rf, model_tf, model_lgb]
labels = ['Random Forest', 'Multi layer Perceptron', 'LightGBM']

# Calculate the average AUC values for each model
average_auc_values = [np.mean(auc_scores_test[i]) for i in range(len(models))]

# Plot the AUC-ROC curve with average AUC values for individual models
plt.figure(figsize=(8, 6))
for i in range(len(models)):
    fpr, tpr, _ = roc_curve(y_test, models[i].predict_proba(X_test)[:, 1])
    roc_auc = average_auc_values[i]
    plt.plot(fpr, tpr, label=f'{labels[i]} (AUC = {roc_auc:.4f})')

# Plot the AUC-ROC curve for the stacked model
fpr_stacked, tpr_stacked, _ = roc_curve(y_test, stacked_model.predict_proba(X_test)[:, 1])
roc_auc_stacked = cross_val_auroc_test
plt.plot(fpr_stacked, tpr_stacked, label=f'Stacked Model (AUC = {roc_auc_stacked:.4f})')

# Plot the AUC-ROC curve for etoxpred
fpr_etoxpred, tpr_etoxpred, _ = roc_curve(y_test, etoxpred_probabilities)
roc_auc_etoxpred = auc(fpr_etoxpred, tpr_etoxpred)
plt.plot(fpr_etoxpred, tpr_etoxpred, label=f'etoxpred (AUC = {roc_auc_etoxpred:.4f})', linestyle='--')



plt.plot([0, 1], [0, 1], 'k--')
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.xlabel('False Positive Rate',fontsize=14)
plt.ylabel('True Positive Rate',fontsize=14)
plt.title('Comparison of Receiver Operating Characteristic for Test Set',fontsize=14)
plt.legend(loc='lower right')

# Display the plot
#plt.savefig("/home/vinod/asetiya/RSC_rev/plots/AUC_test_bonferroni_balanced.png",dpi=600)
plt.show()


In [ ]:
# for external validation test (180)
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# Compute the false positive rate (FPR), true positive rate (TPR), and AUC for each model
fpr_stacked, tpr_stacked, _ = roc_curve(y_sample, stacked_model.predict_proba(x_sample)[:, 1])
roc_auc_stacked = auc(fpr_stacked, tpr_stacked)

fpr_model_rf, tpr_model_rf, _ = roc_curve(y_sample, model_rf.predict_proba(x_sample)[:, 1])
roc_auc_model_rf = auc(fpr_model_rf, tpr_model_rf)

fpr_model_tf, tpr_model_tf, _ = roc_curve(y_sample, model_tf.predict_proba(x_sample)[:, 1])
roc_auc_model_tf = auc(fpr_model_tf, tpr_model_tf)

fpr_model_lgb, tpr_model_lgb, _ = roc_curve(y_sample, model_lgb.predict_proba(x_sample)[:, 1])
roc_auc_model_lgb = auc(fpr_model_lgb, tpr_model_lgb)

# Plot the AUC-ROC curve
plt.figure(figsize=(8, 6))

plt.plot(fpr_model_rf, tpr_model_rf, label='Random Forest (AUC = {:.4f})'.format(roc_auc_model_rf))
plt.plot(fpr_model_tf, tpr_model_tf, label='Multi layer Perceptron(AUC = {:.4f})'.format(roc_auc_model_tf))
plt.plot(fpr_model_lgb, tpr_model_lgb, label='LightGBM (AUC = {:.4f})'.format(roc_auc_model_lgb))
plt.plot(fpr_stacked, tpr_stacked, label='Stacked Model (AUC = {:.4f})'.format(roc_auc_stacked))

# Plot the AUC-ROC curve for etoxpred
fpr_etoxpred, tpr_etoxpred, _ = roc_curve(y_sample, etoxpred_probabilities_ext)
roc_auc_etoxpred = auc(fpr_etoxpred, tpr_etoxpred)
plt.plot(fpr_etoxpred, tpr_etoxpred, label=f'etoxpred (AUC = {roc_auc_etoxpred:.4f})', linestyle='--')


plt.plot([0, 1], [0, 1], 'k--')
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.xlabel('False Positive Rate',fontsize=14)
plt.ylabel('True Positive Rate',fontsize=14)
plt.title('Comparison of Receiver Operating Characteristic for external validation set',fontsize=14)
plt.legend()
#plt.savefig("/home/vinod/asetiya/RSC_rev/plots/ROC_AUC_external_validation_bonferroni.png",dpi=600)
plt.show()